<a href="https://colab.research.google.com/github/NtimTim/DS_School_Home_Work/blob/main/Hometask/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_4_Empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/4vel/python-intro/blob/main/Hometask/Task3_dmrf_OOP.ipynb)

**Практика**

Используемые библиотеки


In [44]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
from itertools import groupby
import re

**Загрузка DataFrame**

**Задача 1 **

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [2]:
# не сработало, TimeoutError
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


ConnectTimeout: HTTPSConnectionPool(host='data.gov.ru', port=443): Max retries exceeded with url: /opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7a4dd2bf6470>, 'Connection to data.gov.ru timed out. (connect timeout=None)'))

In [32]:
### Если не работает загрузка on-line TimeoutError, то в colab так же можно выбрать файл с диска
from google.colab import files
uploaded = files.upload() #выберите файл на диске Химический анализ родника в Нескучном саду.csv
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';', index_col=0)
display(df)

Saving Химический анализ родника в Нескучном саду.csv to Химический анализ родника в Нескучном саду (4).csv


,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [124]:
def check_result(result, norma):

    norma = norma.replace(',', '.')
    ntype = norma.split()[:2]
    nnum = norma.split()[2:]
    nnum1 = nnum[0].split('-')[:1]
    nnum2 = nnum[0].split('-')[1:]

    if result == 'б/цвета':
     result = '0'

#    return nnum2

    if ((ntype[0] == 'в') and (ntype[1] == 'пределах')):
       if ((float(result)>=float(str(nnum1[0])))and(float(result)<=float(str(nnum2[0])))):
         return 'Норма'
       else:
         return 'Отклонение от нормы'
    elif ((ntype[0] == 'не') and (ntype[1] == 'более')):
          if ((float(result)<=float(str(nnum1[0])))):
           return 'Норма'
          else:
           return 'Отклонение от нормы'
    else:
       return 'Обработка результата по нормативу невозможна'

df['Вывод'] = df.apply(lambda row: check_result(row['Результат анализа'], row['Норматив']), axis=1)
display(df)

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,б/цвета,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


**Теория вероятности. События**

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

**Задача 2**

В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

**Задача 3**

Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение